In [116]:
# !pip install fasttext
# Word Embedding using FastText
import fasttext.util
fasttext.util.download_model('ar', if_exists='ignore')  
ft = fasttext.load_model('cc.ar.300.bin')

In [117]:
print(ft.get_dimension())
# reduce the dimension of the word embedding from 300 to 100
fasttext.util.reduce_model(ft, 100)
print(ft.get_dimension())

300
100


In [118]:
print(ft.get_word_vector('الشوق شوق')) 
print(ft.get_word_vector('الشوق شوق').shape)

[-0.01018494 -0.08594008 -0.03731866 -0.00024127  0.04777642 -0.00496229
  0.06186691  0.02239722 -0.0280445   0.04849716 -0.00795461 -0.02634357
  0.01323329 -0.02175279  0.03655794 -0.01601862 -0.01084875  0.01393491
 -0.03315404  0.0335445   0.01913922 -0.02036594 -0.00192303 -0.00421568
  0.01183395  0.01175373 -0.07322915  0.08777917  0.00460946 -0.05397879
 -0.04420993 -0.01738087 -0.01531424 -0.02648391  0.05191071 -0.06218194
  0.03248445 -0.03084662  0.02021079 -0.03369771 -0.01912928  0.00776838
 -0.02871501  0.00045804  0.01323607  0.01341687  0.03769046 -0.00100606
  0.03073437 -0.02756213  0.02987276  0.00607357 -0.0034104   0.00598877
 -0.04017798 -0.02268266  0.04346691 -0.00163789  0.03622608  0.00370311
 -0.03838527  0.01520885  0.00647131 -0.02734641  0.01213152  0.03940092
  0.04791509 -0.01371275 -0.00985732 -0.00985206  0.04072648 -0.01279194
  0.02949059  0.0199767   0.01000008  0.0091176  -0.03566917  0.03153419
 -0.05023986 -0.02361483  0.04186894  0.01141178 -0

In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from scipy.stats import skew
import matplotlib.pyplot as plt
import numpy as np

In [120]:
df_train = pd.read_csv('./Dataset/cleaned_train.csv')
print(df_train.shape)
df_train.head()

(6988, 3)


,text,category,stance
0,بيل غيتس تلقي لقاح كوفيد19 من غير تصوير ابر و ...,celebrity,1
1,وزير صح حد يوم تحديد هل بمؤتمروا صحفي كان ما ع...,info_news,1
2,قول رح يكونو اد مسؤولي ب لبنان ما وصل لقاح ؟ ا...,info_news,1
3,تركيا . . وزير صح فخر دين قوجة تلقي اول جرع من...,celebrity,1
4,وئام وهاب شتم دول خليجي في كل طل اعلامي تسافه ...,personal,0


In [ ]:
df_dev = pd.read_csv('./Dataset/cleaned_dev.csv')
print(df_dev.shape)
df_dev.head()

In [121]:
MAX_TWEET_LENGTH = -1
for tweet in df_train['cleaned_text']:
  tweet_arr = tweet.split(' ')
  MAX_TWEET_LENGTH = max(MAX_TWEET_LENGTH, len(tweet_arr))
print(MAX_TWEET_LENGTH)
print(pd.Series([len(x.split(' ')) for x in df_train['cleaned_text']]).describe())

137
count    6988.000000
mean       25.166714
std        14.403391
min         2.000000
25%        14.000000
50%        20.000000
75%        36.000000
max       137.000000
dtype: float64


In [122]:
def embedded(text):
  embedded_text = np.zeros((len(text),MAX_TWEET_LENGTH*100))
  print(embedded_text.shape)
  for i,tweet in enumerate(text):
    sentence_embedding = np.array([[]])
    for word in tweet.split(" "):
      sentence_embedding = np.append(sentence_embedding, ft.get_word_vector(word))
    sentence_embedding.resize(MAX_TWEET_LENGTH*100)
    embedded_text[i] = sentence_embedding
  return embedded_text

(6988, 13700)


In [123]:
x_train = embedded(df_train['cleaned_text'])
x_test = embedded(df_dev['cleaned_text'])
y_train_stance = df_train['stance']
y_test_stance = df_dev['stance']
y_train_cat = df_train['category']
y_test_cat = df_dev['category']

print(x_train.shape)
print(x_test.shape)
print(y_train_stance.shape)
print(y_test_stance.shape)
print(y_train_cat.shape)
print(y_test_cat.shape)
print(y_train_stance.value_counts())
print(y_test_stance.value_counts())
print(y_train_cat.value_counts())
print(y_test_cat.value_counts())

(4891, 13700)
(2097, 13700)
(4891,)
(2097,)
(4891,)
(2097,)
 1    3876
 0     708
-1     307
Name: stance, dtype: int64
 1    1662
 0     304
-1     131
Name: stance, dtype: int64
info_news       2542
personal         704
celebrity        669
plan             434
unrelated        216
others           132
requests          82
rumors            55
advice            47
restrictions      10
Name: category, dtype: int64
info_news       1074
personal         321
celebrity        306
plan             172
unrelated        107
others            35
requests          30
rumors            24
advice            20
restrictions       8
Name: category, dtype: int64


In [124]:
# PCA 
# x_train = PCA(n_components=100).fit_transform(x_train)
# PCA_test = PCA(n_components=100).fit(x_test_stance)
# x_test_stance = PCA_test.transform(x_test_stance)

In [125]:
y_train_stance = np.array(y_train_stance)
y_test_stance = np.array(y_test_stance)
y_train_cat = np.array(y_train_cat)
y_test_cat = np.array(y_test_cat)

# Random Forest

In [126]:
# Random Forest on stance
clf = RandomForestClassifier(n_estimators=100, random_state=2002)
clf.fit(x_train, y_train_stance)
y_pred = clf.predict(x_test)
print(classification_report(y_test_stance, y_pred))

              precision    recall  f1-score   support

          -1       0.53      0.07      0.12       131
           0       0.63      0.06      0.10       304
           1       0.80      0.99      0.89      1662

    accuracy                           0.80      2097
   macro avg       0.65      0.37      0.37      2097
weighted avg       0.76      0.80      0.73      2097



In [127]:
# Random Forest on category
clf = RandomForestClassifier(n_estimators=100, random_state=2002)
clf.fit(x_train, y_train_cat)
y_pred = clf.predict(x_test)
print(classification_report(y_test_cat, y_pred))

              precision    recall  f1-score   support

      advice       0.00      0.00      0.00        20
   celebrity       0.88      0.44      0.59       306
   info_news       0.56      0.96      0.70      1074
      others       0.00      0.00      0.00        35
    personal       0.56      0.10      0.17       321
        plan       0.29      0.02      0.04       172
    requests       0.38      0.10      0.16        30
restrictions       0.00      0.00      0.00         8
      rumors       0.00      0.00      0.00        24
   unrelated       0.71      0.09      0.17       107

    accuracy                           0.58      2097
   macro avg       0.34      0.17      0.18      2097
weighted avg       0.57      0.58      0.49      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SVM

In [128]:
# SVM on stance
clf = svm.SVC(kernel='linear', C=1.0, probability=True)
clf.fit(x_train, y_train_stance)
y_pred = clf.predict(x_test)
print(classification_report(y_test_stance, y_pred))

              precision    recall  f1-score   support

          -1       0.21      0.24      0.23       131
           0       0.33      0.31      0.32       304
           1       0.85      0.85      0.85      1662

    accuracy                           0.73      2097
   macro avg       0.46      0.47      0.46      2097
weighted avg       0.73      0.73      0.73      2097



In [129]:
# SVM on category
clf = svm.SVC(kernel='linear', C=1.0, probability=True)
clf.fit(x_train, y_train_cat)
y_pred = clf.predict(x_test)
print(classification_report(y_test_cat, y_pred))

              precision    recall  f1-score   support

      advice       0.00      0.00      0.00        20
   celebrity       0.61      0.65      0.63       306
   info_news       0.63      0.70      0.66      1074
      others       0.03      0.03      0.03        35
    personal       0.46      0.42      0.44       321
        plan       0.21      0.19      0.20       172
    requests       0.26      0.23      0.25        30
restrictions       0.00      0.00      0.00         8
      rumors       0.10      0.04      0.06        24
   unrelated       0.32      0.17      0.22       107

    accuracy                           0.54      2097
   macro avg       0.26      0.24      0.25      2097
weighted avg       0.52      0.54      0.53      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [130]:
x_train = x_train.reshape(x_train.shape[0],MAX_TWEET_LENGTH,100)
x_test = x_test.reshape(x_test.shape[0],MAX_TWEET_LENGTH,100)
print(x_train.shape)
print(x_test.shape)

(4891, 137, 100)
(2097, 137, 100)


In [131]:
y_train_stance = y_train_stance + 1
# squeeze the last dimension of y_train
y_train_stance = np.squeeze(y_train_stance)
print(y_train_stance[0])
print(y_train_stance.shape)
y_train_stance = to_categorical(y_train_stance, 3)
print(y_train_stance.shape)

2
(4891,)
(4891, 3)


In [132]:
def map_category_to_int(category):
    y_cat = np.zeros(category.shape)
    # advice = 0, celebrity = 1, info_news = 2, others = 3, personal = 4, plan = 5, requests = 6, restrictions = 7, rumors = 8, unrelated = 9
    for i,cat in enumerate(category):
        if cat == 'advice':
            y_cat[i] = 0
        elif cat == 'celebrity':
            y_cat[i] = 1
        elif cat == 'info_news':
            y_cat[i] = 2
        elif cat == 'others':
            y_cat[i] = 3
        elif cat == 'personal':
            y_cat[i] = 4
        elif cat == 'plan':
            y_cat[i] = 5
        elif cat == 'requests':
            y_cat[i] = 6
        elif cat == 'restrictions':
            y_cat[i] = 7
        elif cat == 'rumors':
            y_cat[i] = 8
        elif cat == 'unrelated':
            y_cat[i] = 9
    return y_cat


print(y_train_cat[0])
y_train_cat_int = map_category_to_int(y_train_cat)
print(y_train_cat_int[0])
y_train_cat_int = to_categorical(y_train_cat_int, 10)
print(y_train_cat_int.shape)

info_news
2.0
(4891, 10)


In [133]:
def map_int_to_category(y_cat):
    category = []
    for i,cat in enumerate(y_cat):
        if cat == 0:
            category.append('advice')
        elif cat == 1:
            category.append('celebrity')
        elif cat == 2:
            category.append('info_news')
        elif cat == 3:
            category.append('others')
        elif cat == 4:
            category.append('personal')
        elif cat == 5:
            category.append('plan')
        elif cat == 6:
            category.append('requests')
        elif cat == 7:
            category.append('restrictions')
        elif cat == 8:
            category.append('rumors')
        elif cat == 9:
            category.append('unrelated')
    return category

# RNN

In [134]:
# RNN model for stance
model1_stance = Sequential()
# model.add(Input(shape=(137,100)))
model1_stance.add(SimpleRNN(units = 100,input_shape=(MAX_TWEET_LENGTH,100),return_sequences=True))
model1_stance.add(Dropout(0.2))
model1_stance.add(SimpleRNN(units = 32))
model1_stance.add(Dropout(0.2))
model1_stance.add(Dense(3, activation='softmax'))
print(model1_stance.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_14 (SimpleRNN)   (None, 137, 100)          20100     
                                                                 
 dropout_20 (Dropout)        (None, 137, 100)          0         
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 32)                4256      
                                                                 
 dropout_21 (Dropout)        (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 3)                 99        
                                                                 
Total params: 24,455
Trainable params: 24,455
Non-trainable params: 0
_________________________________________________________________
None


In [135]:
opt = Adam(learning_rate=0.0001)
model1_stance.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy', 'Precision', 'Recall'])
model1_stance.fit(x_train, y_train_stance, epochs=12, batch_size=32, verbose=1)

Epoch 1/12
153/153 [==============================] - 8s 39ms/step - loss: 0.6907 - accuracy: 0.7804 - precision: 0.7886 - recall: 0.7636
Epoch 2/12
153/153 [==============================] - 6s 39ms/step - loss: 0.6619 - accuracy: 0.7925 - precision: 0.7931 - recall: 0.7906
Epoch 3/12
153/153 [==============================] - 6s 37ms/step - loss: 0.6448 - accuracy: 0.7915 - precision: 0.7983 - recall: 0.7800
Epoch 4/12
153/153 [==============================] - 6s 38ms/step - loss: 0.6331 - accuracy: 0.7894 - precision: 0.8017 - recall: 0.7737
Epoch 5/12
153/153 [==============================] - 6s 38ms/step - loss: 0.6127 - accuracy: 0.7929 - precision: 0.8113 - recall: 0.7751
Epoch 6/12
153/153 [==============================] - 6s 42ms/step - loss: 0.6024 - accuracy: 0.7937 - precision: 0.8099 - recall: 0.7681
Epoch 7/12
153/153 [==============================] - 6s 40ms/step - loss: 0.5930 - accuracy: 0.7955 - precision: 0.8097 - recall: 0.7684
Epoch 8/12
153/153 [==============

In [136]:
y_pred = model1_stance.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = y_pred - 1
print(classification_report(y_test_stance, y_pred))

66/66 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       131
           0       0.64      0.03      0.06       304
           1       0.80      1.00      0.89      1662

    accuracy                           0.79      2097
   macro avg       0.48      0.34      0.31      2097
weighted avg       0.72      0.79      0.71      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [137]:
# RNN model for category
model1_cat = Sequential()
# model.add(Input(shape=(137,100)))
model1_cat.add(SimpleRNN(units = 100,input_shape=(MAX_TWEET_LENGTH,100),return_sequences=True))
model1_cat.add(Dropout(0.2))
model1_cat.add(SimpleRNN(units = 32))
model1_cat.add(Dropout(0.2))
model1_cat.add(Dense(10, activation='softmax'))
print(model1_cat.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_16 (SimpleRNN)   (None, 137, 100)          20100     
                                                                 
 dropout_22 (Dropout)        (None, 137, 100)          0         
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 32)                4256      
                                                                 
 dropout_23 (Dropout)        (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 10)                330       
                                                                 
Total params: 24,686
Trainable params: 24,686
Non-trainable params: 0
_________________________________________________________________
None


In [138]:
opt = Adam(learning_rate=0.0001)
model1_cat.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy', 'Precision', 'Recall'])
model1_cat.fit(x_train, y_train_cat_int, epochs=12, batch_size=32, verbose=1)

Epoch 1/12
153/153 [==============================] - 8s 38ms/step - loss: 1.8850 - accuracy: 0.4484 - precision: 0.5065 - recall: 0.0556
Epoch 2/12
153/153 [==============================] - 6s 38ms/step - loss: 1.6919 - accuracy: 0.5113 - precision: 0.5329 - recall: 0.2118
Epoch 3/12
153/153 [==============================] - 6s 42ms/step - loss: 1.6341 - accuracy: 0.5095 - precision: 0.5216 - recall: 0.2687
Epoch 4/12
153/153 [==============================] - 6s 38ms/step - loss: 1.6177 - accuracy: 0.5083 - precision: 0.5165 - recall: 0.2525
Epoch 5/12
153/153 [==============================] - 6s 38ms/step - loss: 1.5985 - accuracy: 0.5066 - precision: 0.5160 - recall: 0.2768
Epoch 6/12
153/153 [==============================] - 6s 38ms/step - loss: 1.5877 - accuracy: 0.5120 - precision: 0.5193 - recall: 0.2672
Epoch 7/12
153/153 [==============================] - 6s 38ms/step - loss: 1.5927 - accuracy: 0.5132 - precision: 0.5078 - recall: 0.2730
Epoch 8/12
153/153 [==============

In [139]:
y_pred = model1_cat.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = map_int_to_category(y_pred)
print(classification_report(y_test_cat, y_pred))

66/66 [==============================] - 1s 15ms/step
              precision    recall  f1-score   support

      advice       0.00      0.00      0.00        20
   celebrity       0.00      0.00      0.00       306
   info_news       0.54      0.92      0.68      1074
      others       0.00      0.00      0.00        35
    personal       0.33      0.26      0.29       321
        plan       0.00      0.00      0.00       172
    requests       0.00      0.00      0.00        30
restrictions       0.00      0.00      0.00         8
      rumors       0.00      0.00      0.00        24
   unrelated       0.00      0.00      0.00       107

    accuracy                           0.51      2097
   macro avg       0.09      0.12      0.10      2097
weighted avg       0.33      0.51      0.39      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# LSTM

In [140]:
model2_stance = Sequential()
# model.add(Input(shape=(137,100)))
model2_stance.add(LSTM(units = 100,input_shape=(137,100),return_sequences=True))
model2_stance.add(Dropout(0.2))
model2_stance.add(LSTM(units = 32))
model2_stance.add(Dropout(0.2))
model2_stance.add(Dense(3, activation='softmax'))
print(model2_stance.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 137, 100)          80400     
                                                                 
 dropout_24 (Dropout)        (None, 137, 100)          0         
                                                                 
 lstm_7 (LSTM)               (None, 32)                17024     
                                                                 
 dropout_25 (Dropout)        (None, 32)                0         
                                                                 
 dense_12 (Dense)            (None, 3)                 99        
                                                                 
Total params: 97,523
Trainable params: 97,523
Non-trainable params: 0
_________________________________________________________________
None


In [141]:
opt = Adam(learning_rate=0.000001)
model2_stance.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy', 'Precision', 'Recall'])
model2_stance.fit(x_train, y_train_stance, epochs=20, batch_size=32, verbose=2)

Epoch 1/20
153/153 - 22s - loss: 1.0980 - accuracy: 0.7890 - precision: 0.0000e+00 - recall: 0.0000e+00 - 22s/epoch - 141ms/step
Epoch 2/20
153/153 - 18s - loss: 1.0966 - accuracy: 0.7925 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 116ms/step
Epoch 3/20
153/153 - 18s - loss: 1.0952 - accuracy: 0.7923 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 117ms/step
Epoch 4/20
153/153 - 18s - loss: 1.0937 - accuracy: 0.7923 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 118ms/step
Epoch 5/20
153/153 - 18s - loss: 1.0921 - accuracy: 0.7923 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 117ms/step
Epoch 6/20
153/153 - 18s - loss: 1.0902 - accuracy: 0.7925 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 118ms/step
Epoch 7/20
153/153 - 18s - loss: 1.0883 - accuracy: 0.7925 - precision: 0.0000e+00 - recall: 0.0000e+00 - 18s/epoch - 118ms/step
Epoch 8/20
153/153 - 18s - loss: 1.0862 - accuracy: 0.7923 - precision: 0.0000e+00 - recall: 0.00

In [142]:
y_pred = model2_stance.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = y_pred - 1
print(classification_report(y_test_stance, y_pred))

66/66 [==============================] - 4s 46ms/step
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       131
           0       0.00      0.00      0.00       304
           1       0.79      1.00      0.88      1662

    accuracy                           0.79      2097
   macro avg       0.26      0.33      0.29      2097
weighted avg       0.63      0.79      0.70      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [143]:
model2_cat = Sequential()
# model.add(Input(shape=(137,100)))
model2_cat.add(LSTM(units = 100,input_shape=(137,100),return_sequences=True))
model2_cat.add(Dropout(0.2))
model2_cat.add(LSTM(units = 32))
model2_cat.add(Dropout(0.2))
model2_cat.add(Dense(10, activation='softmax'))
print(model2_cat.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 137, 100)          80400     
                                                                 
 dropout_26 (Dropout)        (None, 137, 100)          0         
                                                                 
 lstm_9 (LSTM)               (None, 32)                17024     
                                                                 
 dropout_27 (Dropout)        (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 10)                330       
                                                                 
Total params: 97,754
Trainable params: 97,754
Non-trainable params: 0
_________________________________________________________________
None


In [144]:
opt = Adam(learning_rate=0.000001)
model2_cat.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy', 'Precision', 'Recall'])
model2_cat.fit(x_train, y_train_cat_int, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
153/153 [==============================] - 21s 108ms/step - loss: 2.3020 - accuracy: 0.5171 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/20
153/153 [==============================] - 17s 109ms/step - loss: 2.3009 - accuracy: 0.5197 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/20
153/153 [==============================] - 17s 112ms/step - loss: 2.2996 - accuracy: 0.5197 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/20
153/153 [==============================] - 16s 107ms/step - loss: 2.2983 - accuracy: 0.5197 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/20
153/153 [==============================] - 16s 107ms/step - loss: 2.2968 - accuracy: 0.5195 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/20
153/153 [==============================] - 17s 108ms/step - loss: 2.2953 - accuracy: 0.5197 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/20
153/153 [==============================] - 17s 108ms/step - loss: 2.2936 - accuracy: 0.5195 - precisi

In [145]:
y_pred = model2_cat.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = map_int_to_category(y_pred)
print(classification_report(y_test_cat, y_pred))

66/66 [==============================] - 3s 36ms/step
              precision    recall  f1-score   support

      advice       0.00      0.00      0.00        20
   celebrity       0.00      0.00      0.00       306
   info_news       0.51      1.00      0.68      1074
      others       0.00      0.00      0.00        35
    personal       0.00      0.00      0.00       321
        plan       0.00      0.00      0.00       172
    requests       0.00      0.00      0.00        30
restrictions       0.00      0.00      0.00         8
      rumors       0.00      0.00      0.00        24
   unrelated       0.00      0.00      0.00       107

    accuracy                           0.51      2097
   macro avg       0.05      0.10      0.07      2097
weighted avg       0.26      0.51      0.35      2097



c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
